In [1]:
import wandb
import keras
import numpy as np
import pandas as pd
import pickle as pkl
import tensorflow as tf
from database_tools.tools import RecordsHandler

pd.options.plotting.backend = 'plotly'

def read_test_data(data_dir, n_cores):
    handler = RecordsHandler(data_dir=data_dir)

    dataset = handler.read_records(n_cores=n_cores, AUTOTUNE=tf.data.AUTOTUNE)
    test = dataset['test']
    
    ppg = []
    abp = []
    for p, a in test.as_numpy_iterator():
        ppg.append(p)
        abp.append(a)
    ppg = np.array(ppg).reshape(-1, 256)
    abp = np.array(abp).reshape(-1, 256)
    return dict(ppg=ppg, abp=abp)

with open('/home/cam/Documents/database_tools/data-2022-10-28/mimic3/abp_scaler.pkl', 'rb') as f:
    scaler = pkl.load(f)

# run = wandb.init()
# artifact = run.use_artifact('heartfelt/model-registry/Enceladus:v1', type='model')
# artifact_dir = artifact.download()
# wandb.finish()

In [2]:
model = keras.models.load_model('artifacts/model-restful-eon-16:v16/')

test = read_test_data('/home/cam/Documents/database_tools/data-2022-10-28/mimic3/', 10)
true = test['abp']
pred = model.predict(test['ppg']).reshape(-1, 256)

true = scaler.inverse_transform(true).reshape(-1, 256)
pred = scaler.inverse_transform(pred).reshape(-1, 256)

2022-11-02 19:46:54.692101: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 19:46:54.708229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 19:46:54.708425: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 19:46:54.708936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Reading train split.
Reading val split.
Reading test split.


2022-11-02 19:47:03.656563: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2022-11-02 19:47:03.656736: E tensorflow/stream_executor/cuda/cuda_dnn.cc:398] Possibly insufficient driver version: 470.141.3
2022-11-02 19:47:03.656778: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1130 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Graph execution error:

Detected at node 'unet/conv1d/Conv1D' defined at (most recent call last):
    File "/home/cam/miniconda3/envs/tf/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/cam/miniconda3/envs/tf/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/cam/.cache/pypoetry/virtualenvs/database-tools-Qo_l_FwJ-py3.8/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/cam/.cache/pypoetry/virtualenvs/database-tools-Qo_l_FwJ-py3.8/lib/python3.8/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/cam/.cache/pypoetry/virtualenvs/database-tools-Qo_l_FwJ-py3.8/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/cam/.cache/pypoetry/virtualenvs/database-tools-Qo_l_FwJ-p

In [ ]:
from sklearn.metrics import mean_absolute_error

err = []
for t, p in zip(true, pred):
    err.append(mean_absolute_error(t, p))
err = np.array(err)

In [ ]:
pd.Series(err).plot.hist(nbins=50)

In [ ]:
len(err[err < 15]) / len(err)

In [ ]:
len(err[err < 10]) / len(err)

In [ ]:
len(err[err < 5]) / len(err)